# Generate Keyframes
The file generates keyframes used for submap build. The keyframes should be computed by DLIOM directly. In this file, the (fake) keyframes are generated without SLAM algorithm.

In [83]:
import os
import shutil
import yaml
import numpy as np
import matplotlib.pyplot as plt

from tools.fileloader import load_files, load_xyz_rot, load_overlaps


In [84]:
# load data
config_path = '/home/vectr/PycharmProjects/lidar_learning/data/kitti/dataset/config_kitti.yml'
config = yaml.safe_load(open(config_path))

data_root = config['data_root']
sequences = config['sequences']


In [86]:
# load positions
for sequence in sequences:
    poses_path = os.path.join(data_root, sequence, 'poses/poses.txt')
    overlaps_path = os.path.join(data_root, sequence, 'overlaps/overlaps.bin')
    keyframes_path = os.path.join(data_root, sequence, 'keyframes')
    
    if not os.path.exists(keyframes_path):
        os.makedirs(keyframes_path)
    
    xyz, _ = load_xyz_rot(poses_path)
    overlaps = load_overlaps(overlaps_path)
    
    thresh_dist = 30.0
    thresh_overlap = 0.1
    
    num_scans = xyz.shape[0]
    keyframes_indices = []
    
    for i in range(num_scans):
        # add first keyframe directly
        if i == 0:
            keyframes_indices.append(i)
            continue
        
        new_keyframe = True
        for j in keyframes_indices:
            if np.linalg.norm(xyz[i] - xyz[j]) < thresh_dist and overlaps[i, j] > thresh_overlap:
                new_keyframe = False
                break 
        
        if new_keyframe:
            keyframes_indices.append(i)
    
    print(f'Sequence: {sequence}')
    print(f'keyframes: {len(keyframes_indices)}, total: {num_scans}')
    print(f'Avg keyframe interval: {num_scans / len(keyframes_indices)} \n')
    
    # saving keyframes poses, original projected pngs, indices 
    png512_files = load_files(os.path.join(data_root, sequence, 'png_files/512/orig'))
    png1024_files = load_files(os.path.join(data_root, sequence, 'png_files/1024/orig'))
    
    poses_kf = np.loadtxt(poses_path)[keyframes_indices]
    png512_files_kf = [png512_files[i] for i in keyframes_indices]
    png1024_files_kf = [png1024_files[i] for i in keyframes_indices]
       
    poses_path = os.path.join(keyframes_path, 'poses_kf.txt')
    indices_path = os.path.join(keyframes_path, 'indices_kf.txt')
    png1024_path = os.path.join(keyframes_path, 'png_files/1024')
    png512_path = os.path.join(keyframes_path, 'png_files/512')
    
    if not os.path.exists(png1024_path):
        os.makedirs(png1024_path)
    if not os.path.exists(png512_path):
        os.makedirs(png512_path)
    
    if len(load_files(png512_path)) != 0 or len(load_files(png1024_path)) != 0:   # in case repeat copy
        print(f'Sequence {sequence} keyframes png files already exist!')
        continue
        
    np.savetxt(poses_path, poses_kf, delimiter=' ')
    np.savetxt(indices_path, keyframes_indices, delimiter=' ', fmt='%i')
    
    for f in png512_files_kf:
        shutil.copy(f, png512_path)
    for f in png1024_files_kf:
        shutil.copy(f, png1024_path)      
    

Sequence: 00
keyframes: 93, total: 4541
Avg keyframe interval: 48.82795698924731 

Sequence: 01
keyframes: 79, total: 1101
Avg keyframe interval: 13.936708860759493 

Sequence: 02
keyframes: 150, total: 4661
Avg keyframe interval: 31.073333333333334 

Sequence: 03
keyframes: 19, total: 801
Avg keyframe interval: 42.1578947368421 

Sequence: 04
keyframes: 13, total: 271
Avg keyframe interval: 20.846153846153847 

Sequence: 05
keyframes: 56, total: 2761
Avg keyframe interval: 49.30357142857143 

Sequence: 06
keyframes: 15, total: 1101
Avg keyframe interval: 73.4 

Sequence: 07
keyframes: 22, total: 1101
Avg keyframe interval: 50.04545454545455 

Sequence: 08
keyframes: 91, total: 4071
Avg keyframe interval: 44.73626373626374 

Sequence: 09
keyframes: 54, total: 1591
Avg keyframe interval: 29.462962962962962 

Sequence: 10
keyframes: 30, total: 1201
Avg keyframe interval: 40.03333333333333 

Sequence: 11
keyframes: 28, total: 921
Avg keyframe interval: 32.892857142857146 

Sequence: 12
ke

In [72]:
# visualize keyframes
xyz_kf = xyz[keyframes_indices]

%matplotlib qt
plt.scatter(xyz[:, 0], xyz[:, 1], c='b')
plt.scatter(xyz_kf[:, 0], xyz_kf[:, 1], c='pink')
plt.axis('equal')
plt.show()


dense keyframes sequences: [01, 04, 12, 17, 21]